In [ ]:
import brightway2 as bw
import numpy as np
import json, os

#Local files
from import_databases import *

In [ ]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']
which_pc

### TODO -> give paths to databases files

In [ ]:
if which_pc == 'local':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # Newer databases
    ei371_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.7.1 cutoff/datasets'  
    # ex34_path = '/Users/akim/Documents/LCA_files/exiobase 3.4/IOT_2011_pxp/' #TODO
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
    co_path = 'data/es8b01452_si_002.xlsx'
elif which_pc == 'merlin':
    ex22_path = '/data/user/kim_a/LCA_files/exiobase_22/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ei33_path = '/data/user/kim_a/LCA_files/ecoinvent_33_cutoff/datasets'  
    ei36_path = '/data/user/kim_a/LCA_files/ecoinvent_36_cutoff/datasets'  
    ag13_path = '/data/user/kim_a/LCA_files/agribalyse_13/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/data/user/kim_a/LCA_files/HABE_2017'
    co_path = 'data/es8b01452_si_002.xlsx'

# TODO -> choose project

In [ ]:
project = 'GSA for paper'
bw.projects.set_current(project)
# bw.projects.delete_project(project, delete_dir=True)

ei36_name = 'ecoinvent 3.6 cutoff'
ei371_name = 'ecoinvent 3.7.1 cutoff'
ei_name = ei36_name

In [ ]:
bw.databases

In [ ]:
try:
#     del bw.databases["Agribalyse 1.3 - {} - new biosphere".format(ei_name)]
#     del bw.databases["Agribalyse 1.3 - {}".format(ei_name)]
    del bw.databases["CH consumption 1.0"]
except:
    pass

### Import necessary databases

In [ ]:
%%time 

co_name = CONSUMPTION_DB_NAME

if project == 'GSA for ecoinvent':
    bw.bw2setup()
    import_exiobase_22(ex22_path)
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    import_consumption_db(co_path, habe_path, exclude_dbs=['heia'])
    add_consumption_activities(co_name, habe_path)

elif project == 'GSA for paper':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - ecoinvent 3.6 cutoff',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)
    
elif project == 'GSA no exiobase':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    
elif project == 'GSA for setac':
    bw.bw2setup()
    import_ecoinvent(ei36_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - ecoinvent 3.6 cutoff',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)
    
elif 'GSA for oases' in project:
    bw.bw2setup()
    import_ecoinvent(ei371_path, ei371_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    import_agribalyse_13(ag13_path, ei371_name)
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs, ei_name=ei371_name)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)

In [ ]:
co = bw.Database('CH consumption 1.0')
act = co.search('average consumption')[0]
demand = {act: 1}
method = ('IPCC 2013', 'climate change', 'GWP 100a')
act

In [ ]:
%%time
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

In [ ]:
from bw2calc import MonteCarloLCA
mc = MonteCarloLCA(demand, method)
[next(mc) for _ in range(5)]

# ecoinvent 3.4 changes

In [ ]:
import pandas as pd

In [ ]:
path = "/Users/akim/Downloads/correspondence_file_eiv3.3_to_eiv3.4_20170921_final.xlsx"

In [ ]:
df_raw = pd.read_excel(path, sheet_name="cut-off")

In [ ]:
df = df_raw[13963].dropna()

In [ ]:
for i,a in enumerate(df.values):
    if 'steam' in a:
        print(i,a)

In [ ]:
df_raw.iloc[[11401,11403]]["Unnamed: 31"].values

In [ ]:
df_raw.iloc[11401]